In [2]:
import json
import wandb
import random
import os
import pandas as pd

In [5]:
name = "universenet2008d"  #실험 이름
inference_file = "/opt/ml/detection/baseline/UniverseNet/ensemble/result/submission_ensemble2.csv" #submission file 이름
data_path = "../../dataset/" # dataset 폴더 경로
n = 50 # 몇개의 이미지를 시각화 할지(4000장은 너무 오래 걸려요;;)

wandb.init(project="visualization", entity="boostcampaitech3", name = "univ_ens2")
categories = {0: 'General trash', 1: 'Paper', 2: 'Paper pack', 3: 'Metal', 4: 'Glass', 5: 'Plastic',
 6: 'Styrofoam', 7: 'Plastic bag', 8: 'Battery', 9: 'Clothing'}
labels_file =  pd.read_csv(inference_file)
image_ids = list(labels_file["image_id"])
predictions = list(labels_file["PredictionString"])
imgs=[]

def list_chunk(lst, n):
    return [lst[i:i+n] for i in range(0, len(lst), n)]

for image_id, prediction in zip(image_ids[:n], predictions[:n]):
    wandb_boxes = {"predictions" : {"box_data" : [],
                                    "class_labels" : categories}}
    preds = list_chunk(prediction.split(),6)
    for pred in preds:
        curr_box = {}
        curr_box["position"] = {"minX" : float(pred[2]),
                               "minY" : float(pred[3]),
                               "maxX" : float(pred[4]),
                               "maxY" : float(pred[5])}
        curr_box["class_id"] = int(pred[0])
        curr_box["box_caption"] = categories[int(pred[0])]
        curr_box["scores"] = {"score" : float(pred[1])}
        curr_box["domain"] = "pixel"
        wandb_boxes["predictions"]["box_data"].append(curr_box)

    img_path = os.path.join(data_path, image_id)
    img = wandb.Image(img_path, boxes=wandb_boxes)
    imgs.append(img)
    
wandb.log({"Inference" : imgs})